# sales_raw.csv:

- date
- item_id - Код товара
- qnty - Количество продаж за день

# discounts_raw.csv:

- item_id - Код товара
- promo_type_code - Код типа акции
- sale_price_before_promo - Цена до акции
- sale_price_time_promo - Цена на акцию
- date_start - Дата начала акции
- date_end - Дата окончания акции

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as RMSE
from sklearn.linear_model import LinearRegression

In [2]:
sales_raw = pd.read_csv('/kaggle/input/beltel-ml/sales_raw.csv')

In [3]:
sales_raw['date'] = pd.to_datetime(sales_raw['date'])

In [4]:
sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147143 entries, 0 to 147142
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     147143 non-null  datetime64[ns]
 1   item_id  147143 non-null  int64         
 2   qnty     147143 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.4 MB


In [5]:
discounts_raw = pd.read_csv('/kaggle/input/beltel-ml/discounts_raw.csv')
discounts_raw['date_start'] = pd.to_datetime(discounts_raw['date_start'])
discounts_raw['date_end'] = pd.to_datetime(discounts_raw['date_end'])


In [6]:
discounts_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2081 entries, 0 to 2080
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  2081 non-null   int64         
 1   promo_typeCode           2041 non-null   float64       
 2   sale_price_before_promo  2081 non-null   float64       
 3   sale_price_time_promo    2081 non-null   float64       
 4   date_start               2081 non-null   datetime64[ns]
 5   date_end                 2081 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1)
memory usage: 97.7 KB


In [7]:
discounts_raw.head()

,item_id,promo_typeCode,sale_price_before_promo,sale_price_time_promo,date_start,date_end
0,100551,5.0,159.9,139.9,2023-03-02,2023-03-15
1,100553,5.0,159.9,139.9,2023-03-02,2023-03-15
2,100554,5.0,159.9,139.9,2023-03-02,2023-03-15
3,100558,5.0,159.9,139.9,2023-03-02,2023-03-15
4,100530,2.0,36.9,24.6,2021-09-09,2021-09-22


In [8]:
discounts_raw.groupby(['promo_typeCode']).count()['item_id'].sort_values()

promo_typeCode
1.0       5
7.0       5
23.0      5
16.0      6
2.0       9
44.0     38
8.0      74
10.0    157
6.0     796
5.0     946
Name: item_id, dtype: int64

In [9]:
discounts_raw['promo_typeCode'] = discounts_raw['promo_typeCode'].fillna(5) #Заполнил пустые места значением которое встречается чаще всего

In [10]:
discounts_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2081 entries, 0 to 2080
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  2081 non-null   int64         
 1   promo_typeCode           2081 non-null   float64       
 2   sale_price_before_promo  2081 non-null   float64       
 3   sale_price_time_promo    2081 non-null   float64       
 4   date_start               2081 non-null   datetime64[ns]
 5   date_end                 2081 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1)
memory usage: 97.7 KB


In [11]:
predictions = pd.DataFrame(columns=['date', 'item_id', 'prediction']) #Создание дата-фрейма для прогноза

In [12]:
data_item = sales_raw.loc[sales_raw['date']>'2023-11-30']['item_id'].unique() #Получение всех товаров которые были проданы в декабре 2023

In [13]:
for item_id in data_item:
    df_item_id = sales_raw.loc[sales_raw['item_id']==item_id] #Создаем дата-фрейм только с одним определенным товаром
    df_item_id.reset_index(drop= True , inplace= True )
    #Создание дата-фрейма на январь
    df_01 = pd.DataFrame(columns=['date', 'item_id', 'qnty'])
    for i in pd.date_range('2024-01-01', '2024-01-31'):
        df_01.loc[len(df_01)] = np.array([i, item_id, 0])
    df_01['date'] = pd.to_datetime(df_01['date'])
    df_item_id = pd.concat([df_item_id, df_01]) #Сконкатим дата-фрейм с товаром и дата-фреймом на январь
    
    #Далее добавляем колонки, которые будут в итоговом дата-фрейме (код акции, процент скидки, день недели, месяц и год)
    df_item_id.insert(loc=len(df_item_id.columns), column='promo_typeCode', value = np.array(0)) 
    df_item_id.insert(loc=len(df_item_id.columns), column='sale', value = np.array(0.0))
    df_item_id.insert(loc=len(df_item_id.columns), column='day_of_week', value = df_item_id['date'].dt.dayofweek)
    df_item_id.insert(loc=len(df_item_id.columns), column='month', value = df_item_id['date'].dt.month)
    df_item_id.insert(loc=len(df_item_id.columns), column='year', value = df_item_id['date'].dt.year)

    #Создаем дата-фрейм скидок определенного товара 
    df_discount = discounts_raw[discounts_raw['item_id']==item_id].sort_values(['date_start'])
    df_discount = df_discount.reset_index(drop= True)
    df_discount['sale'] = 1 - df_discount['sale_price_time_promo']/df_discount['sale_price_before_promo'] #Считаем процент скидки
    
    #Далее в цикле по длине дата-фрейма скидок создаем временные интервалы
    #по которым будет заполнятся таблица с товаром
    for i in range(len(df_discount)):
        range_day = pd.date_range(df_discount['date_start'][i], df_discount['date_end'][i]) #Временной интервал скидки 
        #В дата-фрейме продаж товара заполняем соответсвующие колонки
        df_item_id.loc[df_item_id['date'].isin(range_day), 'promo_typeCode'] = df_discount['promo_typeCode'][i]
        df_item_id.loc[df_item_id['date'].isin(range_day), 'sale'] = float(df_discount['sale'][i])
    df_item_id = df_item_id.set_index(df_item_id['date'])
    df_item_id = df_item_id.drop(['date'], axis=1)
    
    #Обучать будем линейную регрессию
    LR = LinearRegression()
    y = df_item_id['qnty'][:'2023-12']
    X = df_item_id.drop(['qnty'], axis = 1)[:'2023-12']
    LR.fit(X, y)
    X_01 = df_item_id.drop(['qnty'], axis = 1)['2024-01':]
    pred = np.sum(LR.predict(X_01))   #Считаем общее количество товара на весь месяц
    predictions.loc[len(predictions)] = ['2024-01-01', item_id, pred]
    
    
    

In [14]:
predictions

,date,item_id,prediction
0,2024-01-01,100545,39.156255
1,2024-01-01,100645,42.873080
2,2024-01-01,100683,32.777254
3,2024-01-01,100612,216.491962
4,2024-01-01,100165,737.913257
...,...,...,...
92,2024-01-01,100474,98.994452
93,2024-01-01,100386,42.017886
94,2024-01-01,100375,5.698590
95,2024-01-01,100046,39.471735


In [15]:
predictions.to_csv('Prediction.csv', sep=';')